In [1]:
import json
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

In [2]:
def load_annotation_json(filepath):
    """
    Загружает JSON-файл с аннотациями.
    
    :param filepath: путь к JSON-файлу
    :return: данные JSON
    """
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

In [3]:
def display_relations_table(data):
    """
    Извлекает связи и отображает их в виде таблицы.
    
    :param data: данные JSON из Label Studio
    :return: pandas.DataFrame со связями
    """
    # Берём первую аннотацию (можно адаптировать под список)
    results = data[0]["annotations"][0]["result"]
    
    # Индексируем сущности по их id
    id_to_entity = {
        item["id"]: {
            "label": item["value"]["labels"][0],
            "text": item["value"]["text"]
        }
        for item in results if item["type"] == "labels"
    }
    
    # Собираем связи
    relations = [
        {
            "from_label": id_to_entity[rel["from_id"]]["label"],
            "from_text": id_to_entity[rel["from_id"]]["text"],
            "relation": rel["labels"][0],
            "to_text": id_to_entity[rel["to_id"]]["text"],
            "to_label": id_to_entity[rel["to_id"]]["label"]
        }
        for rel in results if rel["type"] == "relation"
    ]
    
    df = pd.DataFrame(relations)
    return df

In [4]:
from pyvis.network import Network

def visualize_relations_pyvis(data, output_html="relations_graph.html"):
    df = display_relations_table(data)

    net = Network(height="750px", width="100%", directed=True, notebook=False)
    net.barnes_hut()
    
    added_nodes = set()

    for _, row in df.iterrows():
        from_node = f'{row["from_text"]}\n({row["from_label"]})'
        to_node = f'{row["to_text"]}\n({row["to_label"]})'

        if from_node not in added_nodes:
            net.add_node(from_node, label=from_node)
            added_nodes.add(from_node)
        if to_node not in added_nodes:
            net.add_node(to_node, label=to_node)
            added_nodes.add(to_node)

        net.add_edge(from_node, to_node, label=row["relation"])

    # 👇 Используем write_html без .generate_html()
    net.write_html(output_html, notebook=False, open_browser=True)

In [5]:
path = "project-2-at-2025-03-31-17-29-15d9f73c.json"
data = load_annotation_json(path)

# Таблица связей
df = display_relations_table(data)
print(df)

       from_label          from_text             relation           to_text       to_label
0   review_object  зальные помещения          isLocatedIn  высотных зданиях  building_type
1   review_object  зальные помещения             PROPERTY       числом мест       property
2   review_object  зальные помещения             PROPERTY           высоте        property
3   review_object  зальные помещения             PROPERTY       числом мест       property
4        property        числом мест            MIN_VALUE              300           value
5        property        числом мест            MAX_VALUE              600           value
6        property            высоте   LESS_OR_EQUAL_VALUE               12           value
7           value                12                 UNITS                 м          units
8        property        числом мест           LESS_VALUE              300           value
9        property            высоте   LESS_OR_EQUAL_VALUE               50           value

In [6]:
data = load_annotation_json(path)


# Для интерактивной визуализации (откроется в браузере)
visualize_relations_pyvis(data)